In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
data=pd.read_csv('Data/Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Preprocess the data
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [4]:
X = data.drop("EstimatedSalary", axis=1)
y = data["EstimatedSalary"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
onehot_encoder_gender = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_train_gender_encoded = onehot_encoder_gender.fit_transform(X_train[["Gender"]])
X_test_gender_encoded = onehot_encoder_gender.transform(X_test[["Gender"]])

In [6]:
onehot_encoder_geo = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_train_geo_encoded = onehot_encoder_geo.fit_transform(X_train[["Geography"]])
X_test_geo_encoded = onehot_encoder_geo.transform(X_test[["Geography"]])

In [7]:
gender_encoded_df = pd.DataFrame(X_train_gender_encoded, columns=onehot_encoder_gender.get_feature_names_out(["Gender"]))
gender_encoded_df

,Gender_Female,Gender_Male
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,1.0,0.0
4,0.0,1.0
...,...,...
7995,0.0,1.0
7996,1.0,0.0
7997,1.0,0.0
7998,0.0,1.0


In [8]:
geo_encoded_df = pd.DataFrame(X_train_geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,1.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
7995,1.0,0.0,0.0
7996,1.0,0.0,0.0
7997,1.0,0.0,0.0
7998,1.0,0.0,0.0


In [9]:
gender_train_df = pd.DataFrame(X_train_gender_encoded, 
                           columns=onehot_encoder_gender.get_feature_names_out(["Gender"]),
                           index=X_train.index)
gender_test_df = pd.DataFrame(X_test_gender_encoded,
                          columns=onehot_encoder_gender.get_feature_names_out(["Gender"]),
                          index=X_test.index)

In [10]:
geo_train_df = pd.DataFrame(X_train_geo_encoded, 
                           columns=onehot_encoder_geo.get_feature_names_out(["Geography"]),
                           index=X_train.index)
geo_test_df = pd.DataFrame(X_test_geo_encoded,
                          columns=onehot_encoder_geo.get_feature_names_out(["Geography"]),
                          index=X_test.index)

In [11]:
X_train = pd.concat([X_train.drop(["Geography", "Gender"], axis=1), gender_train_df, geo_train_df], axis=1)
X_test = pd.concat([X_test.drop(["Geography", "Gender"], axis=1), gender_test_df, geo_test_df], axis=1)

In [12]:
X_train

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Gender_Female,Gender_Male,Geography_France,Geography_Germany,Geography_Spain
9254,686,32,6,0.00,2,1,1,0,0.0,1.0,1.0,0.0,0.0
1561,632,42,4,119624.60,2,1,1,0,0.0,1.0,0.0,1.0,0.0
1670,559,24,3,114739.92,1,1,0,1,0.0,1.0,0.0,0.0,1.0
6087,561,27,9,135637.00,1,1,0,1,1.0,0.0,1.0,0.0,0.0
6669,517,56,9,142147.32,1,0,0,1,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,768,54,8,69712.74,1,1,1,0,0.0,1.0,1.0,0.0,0.0
5191,682,58,1,0.00,1,1,1,0,1.0,0.0,1.0,0.0,0.0
5390,735,38,1,0.00,3,0,0,1,1.0,0.0,1.0,0.0,0.0
860,667,43,8,190227.46,1,1,0,1,0.0,1.0,1.0,0.0,0.0


In [13]:
## Scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [14]:
## Save the encoders and scaler
with open ("artifacts/preprocessing/OHE_gender_salary.pkl", "wb") as file:
    pickle.dump(onehot_encoder_gender, file)
with open("artifacts/preprocessing/OHE_geo_salary.pkl", "wb") as file:
    pickle.dump(onehot_encoder_geo, file)
with open('artifacts/preprocessing/scaler_salary.pkl','wb') as file:
    pickle.dump(scaler,file)

#### ANN Regression Problem statement

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [16]:
# Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Output layer for regression
])

## compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                896       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3009 (11.75 KB)
Trainable params: 3009 (11.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# Set up TensorBoard
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [18]:
# Set up Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [19]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[early_stopping_callback, tensorboard_callback]
)

Epoch 1/100


250/250 [==============================] - 5s 10ms/step - loss: 100362.6953 - mae: 100362.6953 - val_loss: 98455.4688 - val_mae: 98455.4688
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 99400.4609 - mae: 99400.4609 - val_loss: 96524.0000 - val_mae: 96524.0000
Epoch 3/100
250/250 [==============================] - 2s 6ms/step - loss: 96078.7422 - mae: 96078.7422 - val_loss: 91684.4297 - val_mae: 91684.4297
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 89683.4297 - mae: 89683.4297 - val_loss: 83876.2266 - val_mae: 83876.2266
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 80692.4375 - mae: 80692.4375 - val_loss: 74281.8984 - val_mae: 74281.8984
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 70741.0156 - mae: 70741.0156 - val_loss: 64952.9062 - val_mae: 64952.9062
Epoch 7/100
250/250 [==============================] - 2s 6ms/step - loss: 61889.1484 - mae: 61889.14

In [20]:
%load_ext tensorboard

In [21]:
%tensorboard --logdir regressionlogs/fit/20260105-184137

In [22]:
## Evaluate model on the test data
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE : {test_mae}')

63/63 [==============================] - 0s 3ms/step - loss: 50409.6914 - mae: 50409.6914
Test MAE : 50409.69140625


In [23]:
model.save('artifacts/model/regression_model.h5')

c:\Prima\Github\Churn_Modeling_Classification_ANN_DLProject\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
